## Launcher notebook

This notebook shows how to launch the flights_model.ipynb notebook either in Deep Learning VM or on Kubeflow pipelines

In [23]:
# change these to try this notebook out
BUCKET = 'hostedkfp-default-bmeiqdunw2'
PROJECT = 'ma-poc-automation'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.5.0/kfp.tar.gz --upgrade

### Create the pipeline

In [16]:
import kfp.components as comp
import kfp.dsl as dsl

# a single-op pipeline that runs the flights pipeline on the pod
@dsl.pipeline(
   name='FlightsPipeline',
   description='Trains, deploys flights model'
)
def flights_pipeline(
    model_name = 'modelname',
    model_version = 'version',
    model_bucket = 'params'
):
    notebookop = dsl.ContainerOp(
      name='flightsmodel',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/submitnotebook:latest',
      arguments=[
        model_bucket
      ],
        file_outputs={'train': '/output.txt'}
    )
    
    deploy_cmle = dsl.ContainerOp(
      name='deploycmle',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/notebook-deploycmle:latest',
      arguments=[
        notebookop.outputs['train'],  # modeldir
        model_name,
        model_version
      ],
      file_outputs={
        'model': '/model.txt',
        'version': '/version.txt'
      }
    )
# compile the pipeline
pipeline_func = flights_pipeline
pipeline_filename = pipeline_func.__name__ + '.tar.gz'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func =pipeline_func, package_path=pipeline_filename)
print(pipeline_filename)

flights_pipeline.tar.gz


In [12]:
pipeline_filename

'flights_pipeline.tar.gz'

In [27]:
GCSDIR='gs://{}/flights/notebook'.format(BUCKET)
arguments = {
     'model_name' : 'flights',
    'model_version' : 'flt_1',
    'model_bucket': BUCKET,
}



In [17]:
import kfp
import kfp.components as comp
client = kfp.Client(host='226b3e95823f7b5e-dot-us-central2.pipelines.googleusercontent.com')

In [19]:
# client.list_pipelines()

In [28]:
run = client.create_run_from_pipeline_func(flights_pipeline, arguments=arguments)

In [2]:
!ls *.tar.gz

flights_pipeline.tar.gz
